# Application usage analysis

In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
df = pd.read_csv('KC_case_data.csv')

In [3]:
df.head(10)

,Unnamed: 0,date,event,purchase_sum,os_name,device_id,utm_source
0,1,2020-01-01,app_start,NaN,ios,833621,vk_ads
1,2,2020-01-01,app_start,NaN,android,1579237,referal
2,3,2020-01-01,app_start,NaN,android,1737182,facebook_ads
3,4,2020-01-01,app_start,NaN,ios,4029024,facebook_ads
4,6,2020-01-01,app_start,NaN,android,8383386,vk_ads
5,7,2020-01-01,app_start,NaN,android,8581267,google_ads
6,9,2020-01-01,app_start,NaN,ios,10274354,instagram_ads
7,10,2020-01-01,app_start,NaN,android,11876266,yandex-direct
8,12,2020-01-01,app_start,NaN,ios,12424502,yandex-direct
9,13,2020-01-01,app_start,NaN,ios,12560979,google_ads


**1. MAU in February:**

In [4]:
df.query('date >= "2020-02-01" and date <= "2020-02-30"').agg({'device_id':'nunique'})

device_id    59808
dtype: int64

**2. Number of installs in January:**

In [5]:
df.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

In [6]:
df.query('date >= "2020-01-01" and date <= "2020-01-31" and event == "app_install"')\
    .agg({'device_id':'nunique'})

device_id    63430
dtype: int64

**3. Assign cohorts to users based on the day the app was installed and calculate the conversion rate from install to purchase for them within 7 days. Which cohort had the highest conversion rate?**

In [7]:
df_query = df.query(' event == "purchase" or event == "app_install"')

In [8]:
df_cohg = df_query.pivot_table(index = ['device_id'], columns='event', values='date', aggfunc='min')\
                .reset_index()

In [9]:
df_cohg['app_install'] = pd.to_datetime(df_cohg['app_install'])

In [10]:
df_cohg['purchase'] = pd.to_datetime(df_cohg['purchase'])

In [11]:
df_cohg['conversion'] = np.where(((df_cohg['purchase'] - df_cohg['app_install'])).isna(),np.nan,
                                 (df_cohg['purchase'] - df_cohg['app_install']) <= datetime.timedelta(days=7))

In [12]:
df_cohg = df_cohg.groupby('app_install')\
                .agg({'device_id': 'count','conversion': 'sum'})

In [13]:
df_cohg['conv_rate'] = df_cohg.conversion/df_cohg.device_id

In [14]:
df_cohg.sort_values('conv_rate', ascending=False)

,device_id,conversion,conv_rate
app_install,,,
2020-01-09,1124,416.0,0.370107
2020-01-14,4054,1491.0,0.367785
2020-01-13,3582,1317.0,0.367672
2020-01-15,3390,1242.0,0.366372
2020-01-19,1803,654.0,0.362729
...,...,...,...
2020-02-21,2047,430.0,0.210064
2020-03-10,487,96.0,0.197125
2020-03-08,863,169.0,0.195829


**4. What paid marketing channel brought the most new users?**

In [16]:
df.query('event == "app_install"').groupby(['utm_source'], as_index=False)\
                                .agg({'os_name':'count'})\
                                .sort_values('os_name', ascending=False)

,utm_source,os_name
5,yandex-direct,25148
1,google_ads,22554
4,vk_ads,19914
2,instagram_ads,17182
0,facebook_ads,11945
3,referal,7947


**5. Analyze at what stage of the funnel most of the registered users fall off**

In [17]:
reg_date = df.query('event == "register"')[['date', 'device_id']]

In [18]:
df_fun = df.merge(reg_date, how='left', on='device_id').rename(columns={'date_x': 'date', 'date_y': 'date_of_reg'})

In [19]:
df_fun.head()

,Unnamed: 0,date,event,purchase_sum,os_name,device_id,utm_source,date_of_reg
0,1,2020-01-01,app_start,NaN,ios,833621,vk_ads,2020-01-01
1,2,2020-01-01,app_start,NaN,android,1579237,referal,2020-01-01
2,3,2020-01-01,app_start,NaN,android,1737182,facebook_ads,2020-01-01
3,4,2020-01-01,app_start,NaN,ios,4029024,facebook_ads,NaN
4,6,2020-01-01,app_start,NaN,android,8383386,vk_ads,2020-01-23


In [20]:
df_fun['reg_or_not'] = df_fun.date_of_reg < df_fun.date

In [21]:
reg_users = df_fun.query('reg_or_not == True')

In [22]:
reg_users.groupby('event')['device_id'].nunique()

event
app_start      25756
choose_item    23804
purchase       15554
search         25419
tap_basket     21684
Name: device_id, dtype: int64

In [23]:
print(f'''Conversion to search from app start {round(reg_users.query('event == "search"')['device_id'].nunique()
        /reg_users.query('event == "app_start"')['device_id'].nunique(),2)}''')

Conversion to search from app start 0.99


In [24]:
print(f'''Conversion to product choosing from search {round(reg_users.query('event == "choose_item"')['device_id'].nunique()
        /reg_users.query('event == "search"')['device_id'].nunique(),2)}''')

Conversion to product choosing from search 0.94


In [25]:
print(f'''Conversion to tap basket from product choosing {round(reg_users.query('event == "tap_basket"')['device_id'].nunique()
        /reg_users.query('event == "choose_item"')['device_id'].nunique(),2)}''')

Conversion to tap basket from product choosing 0.91


In [26]:
print(f'''Conversion to purchase from tap basket {round(reg_users.query('event == "purchase"')['device_id'].nunique()
        /reg_users.query('event == "tap_basket"')['device_id'].nunique(),2)}''')

Conversion to purchase from tap basket 0.72


**6. Users who came from which channels showed the lowest conversion on their first purchase?**

In [27]:
purch = df.query('event == "purchase"')\
            .groupby(['device_id', 'utm_source'])\
            .agg({'date':'min'})\
            .reset_index()\
            .groupby('utm_source')\
            .agg({'device_id':'nunique'})\
            .rename(columns={'device_id':'purchasers'})

In [28]:
start = df.query('event == "app_start"')\
            .groupby(['device_id', 'utm_source'])\
            .agg({'date':'min'})\
            .reset_index()\
            .groupby('utm_source')\
            .agg({'device_id':'nunique'})\
            .rename(columns={'device_id':'clients'})

In [29]:
CR = purch.merge(start, on='utm_source')

In [30]:
CR['cr'] = CR.purchasers/CR.clients

In [31]:
CR.sort_values('cr')

,purchasers,clients,cr
utm_source,,,
yandex-direct,10631,35136,0.302567
google_ads,10004,32837,0.304656
instagram_ads,9500,26915,0.352963
facebook_ads,7965,22513,0.353796
vk_ads,11034,29629,0.372405
referal,5707,13924,0.409868


**7. Users from which channel have a higher median first check? (only first purchases of users are considered)**

In [32]:
date_min = df.query('event == "purchase"')\
                .groupby('device_id')\
                .agg({'date':'min'})

In [33]:
date_purch = df.query('event == "purchase"')[['date','device_id', 'purchase_sum', 'utm_source']]

In [34]:
date_merge = date_min.merge(date_purch, how='left', on=['device_id', 'date'])

In [35]:
date_merge.groupby('utm_source').agg({'purchase_sum':'median'}).sort_values('purchase_sum', ascending=False)

,purchase_sum
utm_source,
instagram_ads,394.5
referal,394.5
vk_ads,394.0
yandex-direct,393.5
google_ads,390.5
facebook_ads,388.5


**8. Which paid acquisition channel has the highest ROMI?**  

Advertising spend data:  
yandex-direct – 10 491 707  
google_ads – 10 534 878  
facebook_ads– 8 590 498  
instagram_ads	 – 8 561 626  
vk_ads	 – 9 553 531  

In [36]:
ROMI = df.groupby('utm_source')\
        .agg({'purchase_sum': 'sum'})\
        .reset_index()

In [38]:
ROMI['market_ex'] = [8590498, 9534878, 8561626, 0, 9553531, 10491707]

In [40]:
ROMI['romi %'] = round((ROMI.purchase_sum - ROMI.market_ex)/ROMI.purchase_sum*100,2)

In [41]:
ROMI.sort_values('romi %', ascending=False)

,utm_source,purchase_sum,market_ex,romi %
3,referal,7305436.0,0,100.00
4,vk_ads,13663424.5,9553531,30.08
2,instagram_ads,12053926.5,8561626,28.97
0,facebook_ads,10067275.5,8590498,14.67
1,google_ads,10705095.0,9534878,10.93
5,yandex-direct,11748818.5,10491707,10.70
